<a href="https://colab.research.google.com/github/kandong54/autodrone/blob/main/model/yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# References

- [YOLOv5](https://github.com/ultralytics/yolov5)
  - [Train Custom Data](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data)
  - [TFLite, ONNX, CoreML, TensorRT Export](https://github.com/ultralytics/yolov5/issues/251)
  - [Transfer Learning with Frozen Layers](https://github.com/ultralytics/yolov5/issues/1314)

# Step 1: Install Requirements

In [ ]:
#clone YOLOv5 and 
!git clone --depth 1 https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt # install dependencies

# #albumentations
# !pip install -q opencv-python-headless==4.5.4.60
# !pip install -q -U albumentations
# !echo "$(pip freeze | grep albumentations) is successfully installed"

# Weights & Biases
!pip install wandb -qqq
import wandb
wandb.login()

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Step 2: Download Dataset


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Download dataset and untar it. The dataset is split into parts due to [googlecolab/colabtools#1915](https://github.com/googlecolab/colabtools/issues/1915).

This takes about 15 minutes.

In [ ]:
%%bash
rm -rf dataset
mkdir dataset
tar -xf /content/drive/Shareddrives/AutoDrone/Datasets/open-images-v6/yolov5/yolov5.tar -C dataset
mkdir dataset/images/train
tar -xf /content/drive/Shareddrives/AutoDrone/Datasets/open-images-v6/yolov5/train0.tar -C dataset/images/train
tar -xf /content/drive/Shareddrives/AutoDrone/Datasets/open-images-v6/yolov5/train1.tar -C dataset/images/train
tar -xf /content/drive/Shareddrives/AutoDrone/Datasets/open-images-v6/yolov5/train2.tar -C dataset/images/train
tar -xf /content/drive/Shareddrives/AutoDrone/Datasets/open-images-v6/yolov5/train3.tar -C dataset/images/train
cp /content/drive/Shareddrives/AutoDrone/Datasets/open-images-v6/yolov5/dataset.yaml /content/yolov5/dataset/dataset.yaml

In [ ]:
!rm -rf runs

# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size, -1 for autobatch
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training
- **freeze:** Number of layers to freeze. backbone=10, all=24
- **single-cls** train multi-class data as single-class

In [ ]:
!python train.py --img 640 --batch 64 --epochs 5 --data dataset/dataset.yaml --weights yolov5n.pt --single-cls # --cache "disk" --batch -1 --freeze 24

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

# Export Model
TFlite quantization: float16(default), [int8](https://www.tensorflow.org/lite/performance/post_training_integer_quant)

In [ ]:
#export your model's weights for future use

from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

# tflite
!python export.py --img 640 --data dataset/dataset.yaml --weights runs/train/exp/weights/best.pt --include tflite # --int8
files.download('./runs/train/exp/weights/best-fp16.tflite')
# val tflite
!python val.py --img 640 --data dataset/dataset.yaml --batch-size 64 --weights runs/train/exp/weights/best-fp16.tflite --single-cls --augment

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# !python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source dataset/test/images

In [ ]:
# #display inference on ALL test images

# import glob
# from IPython.display import Image, display

# for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
#     display(Image(filename=imageName))
#     print("\n")

# Conclusion and Next Steps

Congratulations! You've trained a custom YOLOv5 model to recognize your custom objects.

To improve you model's performance, we recommend first interating on your datasets coverage and quality. See this guide for [model performance improvement](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results).

To deploy your model to an application, see this guide on [exporting your model to deployment destinations](https://github.com/ultralytics/yolov5/issues/251).

Once your model is in production, you will want to continually iterate and improve on your dataset and model via [active learning](https://blog.roboflow.com/what-is-active-learning/).